In [1]:
import pandas as pd
import seaborn as sns
import pandas.util.testing as tm

/home/user/.local/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [2]:
df = pd.read_csv('project.csv')
df.head()

,Age,Gender,BMI,Diabetes,Cardio_Vascular_Diseases,Sickle_cell_diesases,Immuno_deficiency_disease,Down_syndrome,Cancer,Chronic_Respiratory_disease,Hypertension,Not Vaccinated,Outcome
0,50,Male,30.1,1,0,0,0,0,0,1,1,1,1
1,31,Male,26.6,1,1,0,0,0,0,1,1,1,1
2,32,Female,23.3,1,1,0,0,0,0,1,1,1,1
3,21,Female,28.1,1,1,0,0,0,0,1,1,1,1
4,33,Male,43.1,1,0,1,0,1,0,1,1,1,1


In [3]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import pickle

In [4]:
x=df.drop(["Outcome"],axis = 1)
y = df['Outcome']

In [5]:
import sklearn
from sklearn.preprocessing import LabelEncoder

In [6]:
x = x.apply(LabelEncoder().fit_transform)

In [7]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=.20, random_state=42)

In [8]:
#Creating the model
logisticRegr = LogisticRegression(C=1)
logisticRegr.fit(x_train, y_train)
y_pred = logisticRegr.predict(x_test)

/home/user/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [9]:
from sklearn.base import BaseEstimator
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [10]:
class ForwardSelectionLinearRegression(BaseEstimator):
    
    def __init__(self):
        self.optimal_predictors_ = None
        self.optimal_model_ = None
        
    def fit(self, X, y=None):
        current_predictors = []
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        remaining_predictors = list(X)
        adjusted_r2_diff = 1
        last_adjusted_r2 = 0
        
        while True:
            best_predictor, adjusted_r2, r2 = self._get_next_best_predictor_(X=X, Y=y, current_predictors = current_predictors, 
                                                              remaining_predictors = remaining_predictors)
            adjusted_r2_diff = round(adjusted_r2 - last_adjusted_r2, 2)
            if adjusted_r2_diff <= 0.00 or len(remaining_predictors) <= 0:
                break
            current_predictors.append(best_predictor)
            remaining_predictors.remove(best_predictor)
            last_adjusted_r2 = adjusted_r2
            
        self.optimal_predictors_ = current_predictors
        self.optimal_model_ = LinearRegression().fit(X[self.optimal_predictors_], y)
        
    def _adjusted_r2_score_(self, r2_val, n, k):
        return (1 - (((1- r2_val)*(n-1))/(n-k-1)))
    
    def _get_next_best_predictor_(self, X,Y, current_predictors = None, remaining_predictors = None):
        max_adjusted_r2_value = 0
        adjusted_r2_value = 0
        best_r2_val  = 0
        r2Val = 0
        best_predictor = None
        n = len(X)
        
        for predictor in remaining_predictors:
            lr = LinearRegression()
            predictors = []
            predictors.extend(current_predictors)
            predictors.append(predictor)      
            lr.fit(X[predictors], Y)
            predY = lr.predict(X[predictors])
            r2Val = r2_score(Y, predY)
            adjusted_r2_value = self._adjusted_r2_score_(r2Val, n, len(predictors))
            if max_adjusted_r2_value < adjusted_r2_value:
                best_predictor = predictor
                max_adjusted_r2_value = adjusted_r2_value
                best_r2_val = r2Val
            
        return best_predictor, max_adjusted_r2_value, best_r2_val

    def predict(self,X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        X = X[self.optimal_predictors_]
        return self.optimal_model_.predict(X)

In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

fs_lr = ForwardSelectionLinearRegression()
fs_lr_pl = Pipeline(steps = [('scaler', StandardScaler()),('forward_selection_model', fs_lr)])

fs_lr_pl.fit(x_train,y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('forward_selection_model',
                 ForwardSelectionLinearRegression())])

In [12]:
list(df.iloc[:,fs_lr.optimal_predictors_])

['Chronic_Respiratory_disease',
 'Not Vaccinated',
 'Down_syndrome',
 'Gender',
 'Age',
 'Hypertension',
 'Cardio_Vascular_Diseases',
 'Diabetes',
 'Sickle_cell_diesases',
 'Cancer']

In [13]:
df.corr()

,Age,BMI,Diabetes,Cardio_Vascular_Diseases,Sickle_cell_diesases,Immuno_deficiency_disease,Down_syndrome,Cancer,Chronic_Respiratory_disease,Hypertension,Not Vaccinated,Outcome
Age,1.000000,0.034975,-0.010832,0.038261,-0.017623,0.036467,0.041859,0.020102,-0.013268,0.046041,0.083114,0.279008
BMI,0.034975,1.000000,0.011190,0.032059,0.012285,-0.042304,-0.001082,-0.037018,0.047454,-0.057766,0.004903,0.018888
Diabetes,-0.010832,0.011190,1.000000,-0.212802,-0.269162,-0.056820,0.011345,-0.267945,-0.062354,-0.059445,-0.022732,0.064728
Cardio_Vascular_Diseases,0.038261,0.032059,-0.212802,1.000000,-0.116540,-0.044265,0.000564,-0.123784,-0.036678,-0.031071,-0.015486,0.104784
Sickle_cell_diesases,-0.017623,0.012285,-0.269162,-0.116540,1.000000,0.352154,0.513492,0.375019,0.031971,-0.057435,-0.042154,0.233383
Immuno_deficiency_disease,0.036467,-0.042304,-0.056820,-0.044265,0.352154,1.000000,0.718279,0.485507,-0.030673,-0.033354,0.043572,0.212040
Down_syndrome,0.041859,-0.001082,0.011345,0.000564,0.513492,0.718279,1.000000,0.335986,0.029295,-0.000418,0.057306,0.311160
Cancer,0.020102,-0.037018,-0.267945,-0.123784,0.375019,0.485507,0.335986,1.000000,0.115386,0.114465,0.045460,0.196033
Chronic_Respiratory_disease,-0.013268,0.047454,-0.062354,-0.036678,0.031971,-0.030673,0.029295,0.115386,1.000000,0.290002,-0.083235,0.391594
Hypertension,0.046041,-0.057766,-0.059445,-0.031071,-0.057435,-0.033354,-0.000418,0.114465,0.290002,1.000000,0.287232,0.324378


In [14]:
import streamlit as st
import pandas as pd

In [18]:
Age= int(input("Age:"))
Sex=int(input("Sex:"))
BMI=float(input("BMI:"))
Diabetes=int(input("Diabetes:"))
Cardio_Vascular_Diseases=int(input("Cardio Vascular Diseases:"))
Sickle_cell_diesases=int(input("Sickle cell diesases:"))
Immuno_deficiency_diseases=int(input("Immuno deficiency diseases:"))
Down_syndrome=int(input("Down syndrome:"))
Cancer=int(input("Cancer:"))
Chronic_Respiratory_disease=int(input("Chronic Respiratory disease:"))
Hypertension=int(input("Hypertension:"))
Vaccinated=int(input("Vaccinated:"))
New_Data=[[Age,Sex,BMI,Diabetes,Cardio_Vascular_Diseases,Sickle_cell_diesases,Immuno_deficiency_diseases,Down_syndrome,Cancer,Chronic_Respiratory_disease,Hypertension,Vaccinated]]
New_Data_User = pd.DataFrame(New_Data,columns=['Age','Sex','BMI','Diabetes','Cardio_Vascular_Diseases','Sickle_cell_diesases','Immuno_deficiency_diseases','Down_syndrome','Cancer','Chronic_Respiratory_disease','Hypertension','Vaccinated'])
print(New_Data_User.head())
user_classification = logisticRegr.predict(New_Data_User)
print(user_classification)


Age:21
Sex:0
BMI:21
Diabetes:0
Cardio Vascular Diseases:0
Sickle cell diesases:0
Immuno deficiency diseases:0
Down syndrome:0
Cancer:0
Chronic Respiratory disease:0
Hypertension:0
Vaccinated:1
   Age  Sex   BMI  Diabetes  Cardio_Vascular_Diseases  Sickle_cell_diesases  \
0   21    0  21.0         0                         0                     0   

   Immuno_deficiency_diseases  Down_syndrome  Cancer  \
0                           0              0       0   

   Chronic_Respiratory_disease  Hypertension  Vaccinated  
0                            0             0           1  
[0]


In [19]:
if (user_classification<=0):
    print('Congrats,You are under Low risk of affecting Corona')
else:
    print(" we are really sorry to say but it seems like You are under High risk of affecting Corona.")

Congrats,You are under Low risk of affecting Corona


In [17]:
#Saving the Model
pickle_out = open("logisticRegr.pkl", "wb") 
pickle.dump(logisticRegr, pickle_out) 
pickle_out.close()